In [1]:
import os
script_name = 'Test_20180928.ipynb'
dir_path = os.path.dirname(os.path.realpath(script_name))
parent_path = os.path.dirname(dir_path)
Func_path = os.path.join(parent_path,'Func')

import sys
sys.path.append(Func_path)

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from loading_data import DataLoader_base
from building_network import NetworkBuild_base
from run_network import NetworkRun_base

Using TensorFlow backend.


In [3]:
dataset = DataLoader_base(data_source="inside",filename="cifar10")

In [4]:
print(dataset.info)

{'validation_status': True, 'input_shape': (32, 32, 3), 'num_train': 50000, 'output_shape': (1,), 'num_test': 10000, 'dataset_name': 'cifar10'}


In [5]:
network = NetworkBuild_base(task="classification", input_shape=[32,32,3], 
                            output_shape = 10, pre_train=False, net_name="VGG19", customized_model=None)

In [6]:
print(network.model.summary())
print(len(network.model.layers))



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

In [7]:
import keras
dataset.output_train = keras.utils.to_categorical(dataset.output_train, network.output_shape)
dataset.output_test = keras.utils.to_categorical(dataset.output_test, network.output_shape)
dataset.input_train = dataset.input_train.astype('float32')
dataset.input_test = dataset.input_test.astype('float32')
dataset.input_train /= 255
dataset.input_test /= 255

In [8]:
train_model = NetworkRun_base(model=network.model, opt = keras.optimizers.SGD(lr=0.1), 
                              loss = keras.losses.mean_squared_error,
                              input_train=dataset.input_train, output_train=dataset.output_train, 
                              batch_size=5,epochs=5,
                              save_dir=os.path.join(parent_path,"SavedModel"), model_name="cifar10_test.h5")

Epoch 1/5
50000/50000 [==============================] - 23s 465us/step - loss: 0.0845
Epoch 2/5
50000/50000 [==============================] - 22s 445us/step - loss: 0.0744
Epoch 3/5
50000/50000 [==============================] - 22s 446us/step - loss: 0.0664
Epoch 4/5
50000/50000 [==============================] - 22s 445us/step - loss: 0.0601
Epoch 5/5
50000/50000 [==============================] - 22s 448us/step - loss: 0.0554
Saved trained model at /home/weixin/Documents/MyProjects/DL_script/SavedModel/cifar10_test.h5 


In [9]:
scores = network.model.evaluate(dataset.input_test, dataset.output_test, verbose=1)
print(scores)

10000/10000 [==============================] - 1s 78us/step
0.0493688617587


In [10]:
scores = train_model.model.evaluate(dataset.input_test, dataset.output_test, verbose=1)
print(scores)

10000/10000 [==============================] - 1s 77us/step
0.0493688617587


In [ ]:
from keras.models import load_model
save_dir=os.path.join(parent_path,"SavedModel") model_name="cifar10_test.h5"
model = load_model(os.path.join(train_model.save_dir,train_model.model_name))

In [ ]:
tmp_model = Sequential()
tmp_model.add(Input(shape=[224,224,500]))
if network.model.output_shape[-1] == 512:
    tmp_model.add(network.model.layers[-1])
    network.model.layers.pop()

In [ ]:
from keras.models import Model
from keras.layers import Input
sub_model_1 = Input(shape=[100,100,512])
sub_model_2 = network.model.layers[-1]
new_model = Model(inputs=sub_model_1,outputs=sub_model_2.output)
print(new_model.summary())

In [ ]:
dataset = DataLoader_base(data_source="inside",filename="mnist")

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
	def BuildModel(input_shape,output_shape):
		model = Sequential()
		model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=input_shape))
		model.add(Activation('relu'))
		model.add(Conv2D(32, (3, 3)))
		model.add(Activation('relu'))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		model.add(Conv2D(64, (3, 3), padding='same'))
		model.add(Activation('relu'))
		model.add(Conv2D(64, (3, 3)))
		model.add(Activation('relu'))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation('relu'))
		model.add(Dropout(0.5))
		model.add(Dense(output_shape))
		model.add(Activation('softmax'))
		return model

In [ ]:
del tmp
tmp = BuildModel(network.input_shape,network.output_shape)

In [ ]:
print(tmp.summary())